In [2]:
###############################################################################
# This notebook was written to process, cleanup and attempt to retrieve       #
# better student ID and school ID data from MIEMIS on a request from Poon     #
# from Pacific Testing Center (OnlineSBA)                                     #
# An ad-hoc support request that led to a more general and re-usable notebook #
# soe-to-onlinesba.ipynb                                                      #
###############################################################################

# Import core stuff
import json

# Import Data stuff
import pandas as pd # Data analysis
import xlrd # excel 
import pyodbc # SQL DB

# fuzz is used to compare TWO strings
from fuzzywuzzy import fuzz
# process is used to compare a string to MULTIPLE other strings
from fuzzywuzzy import process

# Configuration
with open('config.json', 'r') as file:
     config = json.load(file)

# It is important to keep the order of the cells since there are inplace 
# operations on DataFrames

In [3]:
# Process data into a cleanish DataFrame

import os

cwd = os.getcwd()
f = os.path.join(cwd, 'data/RMI/poon-cleanup-request/M03 2019.xlsx')

df_exams = pd.read_excel(f, index_col=None, header=0, engine='openpyxl')
df_exams.dropna(how='all',inplace=True)
df_exams.reset_index(drop=True, inplace=True)
df_exams


# investigate [nan, 'MH010787', None, 'MH009285', 'MH035753'] # They exists here!!!
#df_exams[df_exams['STUDENTNAME'] == 'Rine Sam']
#df_exams[df_exams['STUDENTNAME'] == 'Rine Sam'].STUDENTNAME

#len(df_exams['STUDENTID'].unique())

,STUDENTID,SPED,ACCOM,STUDENTNAME,SCHOOLID,GENDER,TESTID,TEACHERNAME,ITEM_001,ITEM_002,...,ITEM_024,ITEM_025,ITEM_026,ITEM_027,ITEM_028,ITEM_029,ITEM_030,ITEM_031,ITEM_032,END
0,1.0,NO,NO,Biten Tarkij,NaN,NaN,M03,Imelda Schutz,A,A,...,C,D,D,D,D,D,D,D,D,NaN
1,2.0,NO,NO,Ainet Milne,NaN,NaN,M03,Isabella Albert,A,A,...,A,D,A,C,B,A,C,B,D,NaN
2,3.0,NO,NO,Keity William,NaN,NaN,M03,Romine Joel,A,D,...,D,D,A,C,A,D,A,B,D,NaN
3,4.0,NO,NO,Samson Kios,NaN,NaN,M03,Romine Joel,A,C,...,B,D,A,C,A,A,C,C,B,NaN
4,5.0,NO,NO,Cuttyno Zilla,NaN,NaN,M03,Romine Joel,D,C,...,B,D,A,B,A,A,C,A,B,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,790.0,NO,NO,Bernice Subille,NaN,NaN,M03,Takmor Johnson,A,C,...,A,D,B,C,A,A,C,D,A,NaN
790,791.0,NO,NO,Deied Dribo,NaN,NaN,M03,Takmor Johnson,A,B,...,D,D,A,B,C,D,C,B,A,NaN
791,792.0,NO,NO,Ailynn Robe,NaN,NaN,M03,Clena Bujen,A,B,...,A,B,A,B,A,B,MULT,MULT,MULT,NaN
792,793.0,NO,NO,Jolynn Sailass,NaN,NaN,M03,Clena Bujen,A,C,...,A,B,A,B,A,B,A,B,A,NaN


In [5]:
# Load the relevant student enrollments from the database
# there we have all known students loaded from census year after year
enrol_year = 2019

# Establish a database server connection
conn = """
    Driver={{ODBC Driver 17 for SQL Server}};
    Server={},{};
    Database={};
    authentication=SqlPassword;UID={};PWD={};
    TrustServerCertificate=yes;
    autocommit=True
    """.format(config['server_ip'], config['server_port'], config['database'], config['uid'], config['pwd'])

sql_conn = pyodbc.connect(conn)


query = """
SELECT
	stuCardID
	, CONCAT(stuGiven,' ',stuFamilyName) AS Student -- stuMiddleNames,' ',
	, stuGender
	, stuDoB
	, schNo
	, stueYear
	FROM Student_ S
	INNER JOIN StudentEnrolment_ SE ON S.stuID = SE.stuID
	WHERE stueYear = {0}
""".format(enrol_year)
                          
df_student_enrol = pd.read_sql(query, sql_conn)
df_student_enrol

#df_student_enrol.head(3)
#df_student_enrol.count()
#df_student_enrol[df_student_enrol['Student'].str.contains('Ranny George', case=False)]

# investigate [nan, 'MH010787', None, 'MH009285', 'MH035753'] 
# They existed here but pandas does not equate as there were spaces to trim!!!
#df_student_enrol[df_student_enrol['stuCardID'] == 'MH035753']
#df_student_enrol[df_student_enrol['stuCardID'] == 'MH035753'].Student
#df_student_enrol[df_student_enrol['Student'].str.strip() == 'Rine Sam']

,stuCardID,Student,stuGender,stuDoB,schNo,stueYear
0,MH009458,Natasha Wajar,F,2011-03-29,KWA112,2019
1,MH008480,Ruth Bejang,F,2001-06-02,KWA116,2019
2,MH003442,Rusty Jesse,M,2007-06-01,MAJ105,2019
3,MH004155,Mantina Timius,F,2001-08-03,JAL106,2019
4,MH003198,Jacobson Takia,M,2010-03-29,MAJ105,2019
...,...,...,...,...,...,...
14255,MH028489,Mercy Biamon,F,2013-02-04,LAE101,2019
14256,MH028490,Alfred Jibas,M,2013-07-05,LAE101,2019
14257,MH028491,Tommy Branij,M,2013-04-20,LAE101,2019
14258,MH028492,Maita Moiova,F,2013-04-24,LAE101,2019


In [6]:
# Merge both the dirty exams data with the clean student enrollments dataset

   
# lower case to make join case insensitive (like SQL Server, the default collation of Pacific EMIS anyway)
df_exams['STUDENTNAME'] = df_exams['STUDENTNAME'].str.lower()
df_student_enrol['Student'] = df_student_enrol['Student'].str.lower()

# Also need to trim spaces to make it exactly like the SQL Server join
df_exams['STUDENTNAME'] = df_exams['STUDENTNAME'].str.strip()
df_student_enrol['Student'] = df_student_enrol['Student'].str.strip()

#df_exams_and_students = df_exams.set_index('STUDENTNAME').join(df_student_enrol.set_index('Student'), lsuffix='_caller', rsuffix='_other')
df_exams_and_students = df_exams.merge(df_student_enrol, how='left', left_on='STUDENTNAME', right_on='Student', suffixes=('_from_exams', '_from_db'))
df_exams_and_students

#df_exams_and_students[df_exams_and_students[STUDENTID=='Ranny George']]
#df_exams_and_students.loc['Ranny George']
#df_exams_and_students.loc['RANNY GEORGE']
#df_exams_and_students[df_exams_and_students[schNo=='MH000036']]


,STUDENTID,SPED,ACCOM,STUDENTNAME,SCHOOLID,GENDER,TESTID,TEACHERNAME,ITEM_001,ITEM_002,...,ITEM_030,ITEM_031,ITEM_032,END,stuCardID,Student,stuGender,stuDoB,schNo,stueYear
0,1.0,NO,NO,biten tarkij,NaN,NaN,M03,Imelda Schutz,A,A,...,D,D,D,NaN,MH003302,biten tarkij,M,2010-06-14,MAJ105,2019.0
1,2.0,NO,NO,ainet milne,NaN,NaN,M03,Isabella Albert,A,A,...,C,B,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,NO,NO,keity william,NaN,NaN,M03,Romine Joel,A,D,...,A,B,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NO,NO,samson kios,NaN,NaN,M03,Romine Joel,A,C,...,C,C,B,NaN,MH029442,samson kios,M,2009-09-13,UTR101,2019.0
4,5.0,NO,NO,cuttyno zilla,NaN,NaN,M03,Romine Joel,D,C,...,C,A,B,NaN,MH029441,cuttyno zilla,M,2010-07-05,UTR101,2019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,790.0,NO,NO,bernice subille,NaN,NaN,M03,Takmor Johnson,A,C,...,C,D,A,NaN,MH010443,bernice subille,F,2010-05-14,RON101,2019.0
794,791.0,NO,NO,deied dribo,NaN,NaN,M03,Takmor Johnson,A,B,...,C,B,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN
795,792.0,NO,NO,ailynn robe,NaN,NaN,M03,Clena Bujen,A,B,...,MULT,MULT,MULT,NaN,MH010787,ailynn robe,F,2010-05-03,KWA102,2019.0
796,793.0,NO,NO,jolynn sailass,NaN,NaN,M03,Clena Bujen,A,C,...,A,B,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Get the duplicates. This could be that there are two or more matches
# of exams students into the clean DB students enrollments
# (e.g. same name, different student, which one is it?)
#df_exams_and_students_dups = df_exams_and_students[df_exams_and_students.index.duplicated(keep=False)]
df_exams_and_students_dups = df_exams_and_students[df_exams_and_students['STUDENTNAME'].duplicated(keep=False)]
df_exams_and_students_dups

,STUDENTID,SPED,ACCOM,STUDENTNAME,SCHOOLID,GENDER,TESTID,TEACHERNAME,ITEM_001,ITEM_002,...,ITEM_030,ITEM_031,ITEM_032,END,stuCardID,Student,stuGender,stuDoB,schNo,stueYear
54,55.0,NO,NO,roland lang,NaN,NaN,M03,Katol Levai,A,C,...,A,C,D,NaN,MH006536,roland lang,M,2000-12-01,MAJ111,2019.0
55,55.0,NO,NO,roland lang,NaN,NaN,M03,Katol Levai,A,C,...,A,C,D,NaN,MH006536,roland lang,M,2009-07-14,MAJ101,2019.0
444,444.0,NO,NO,mercy capelle,NaN,NaN,M03,Clanny Carlon,B,B,...,C,A,B,NaN,MH013382,mercy capelle,F,2012-03-13,KWA108,2019.0
445,444.0,NO,NO,mercy capelle,NaN,NaN,M03,Clanny Carlon,B,B,...,C,A,B,NaN,MH027779,mercy capelle,F,2010-11-12,KWA108,2019.0
754,753.0,NO,NO,lincoln kabo,NaN,NaN,M03,Salvia Pearson,A,B,...,D,C,B,NaN,MH035959,lincoln kabo,M,2010-01-24,EBO101,2019.0
755,753.0,NO,NO,lincoln kabo,NaN,NaN,M03,Salvia Pearson,A,B,...,D,C,B,NaN,MH035986,lincoln kabo,M,2001-12-10,JAL106,2019.0
781,779.0,NO,NO,ranny george,NaN,NaN,M03,Akiko Lakjohn,A,B,...,C,C,B,NaN,MH000036,ranny george,M,2009-04-16,WTH103,2019.0
782,779.0,NO,NO,ranny george,NaN,NaN,M03,Akiko Lakjohn,A,B,...,C,C,B,NaN,MH002089,ranny george,M,2001-03-07,WTH101,2019.0


In [8]:
# Get the exact matches (i.e. exact name in exams data and the database)
# not actually using this, included for observations. We'll be using fuzzy matching
df_exact_matches = df_exams_and_students.dropna(how='all', subset=['stuCardID']) #subset=['stuCardID', 'stuGender', 'stuDoB', 'schNo', 'stueYear'])
df_exact_matches

,STUDENTID,SPED,ACCOM,STUDENTNAME,SCHOOLID,GENDER,TESTID,TEACHERNAME,ITEM_001,ITEM_002,...,ITEM_030,ITEM_031,ITEM_032,END,stuCardID,Student,stuGender,stuDoB,schNo,stueYear
0,1.0,NO,NO,biten tarkij,NaN,NaN,M03,Imelda Schutz,A,A,...,D,D,D,NaN,MH003302,biten tarkij,M,2010-06-14,MAJ105,2019.0
3,4.0,NO,NO,samson kios,NaN,NaN,M03,Romine Joel,A,C,...,C,C,B,NaN,MH029442,samson kios,M,2009-09-13,UTR101,2019.0
4,5.0,NO,NO,cuttyno zilla,NaN,NaN,M03,Romine Joel,D,C,...,C,A,B,NaN,MH029441,cuttyno zilla,M,2010-07-05,UTR101,2019.0
5,6.0,NO,NO,kimilynn kenes,NaN,NaN,M03,Romine Joel,A,C,...,C,A,B,NaN,MH001549,kimilynn kenes,F,2009-11-24,UTR101,2019.0
6,7.0,NO,NO,jimiko sammy,NaN,NaN,M03,Romine Joel,A,C,...,B,A,D,NaN,MH001546,jimiko sammy,F,2010-05-15,UTR101,2019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,785.0,NO,NO,orlando debrum,NaN,NaN,M03,Takmor Johnson,A,B,...,B,C,D,NaN,MH008197,orlando debrum,M,2010-01-17,MAJ121,2019.0
790,787.0,NO,NO,yostami compass,NaN,NaN,M03,Takmor Johnson,A,MULT,...,B,D,A,NaN,MH010447,yostami compass,M,2010-08-13,RON101,2019.0
792,789.0,NO,NO,aliann shem,NaN,NaN,M03,Takmor Johnson,A,C,...,D,B,C,NaN,MH010441,aliann shem,F,2009-11-09,RON101,2019.0
793,790.0,NO,NO,bernice subille,NaN,NaN,M03,Takmor Johnson,A,C,...,C,D,A,NaN,MH010443,bernice subille,F,2010-05-14,RON101,2019.0


In [9]:
# Troubleshooting differences with SQL
# No longer needed
#query2 = """
#SELECT * FROM [dbo].[StudentMISATData] AS SMISAT LEFT JOIN 
#(SELECT
#	stuCardID
#	, CONCAT(stuGiven,' ',stuFamilyName) AS Student -- stuMiddleNames,' ',
# 	, stuGender
# 	, stuDoB
# 	, schNo
# 	, stueYear
# 	FROM Student_ S
# 	INNER JOIN StudentEnrolment_ SE ON S.stuID = SE.stuID
# 	WHERE stueYear = 2019) AS S ON SMISAT.STUDENTNAME = S.Student
# """   
# df_student_enrol2 = pd.read_sql(query2, sql_conn)
# df_student_enrol2

# # investigate [nan, 'MH010787', None, 'MH009285', 'MH035753']
# s = df_exams_and_students['stuCardID']
# s.isna().sum()
# l = list(s)
# df_exams_and_students[df_exams_and_students['stuCardID'] == 'MH035753']
# df_exams_and_students[df_exams_and_students['Student'] == 'Rine Sam']
# df_exams_and_students[df_exams_and_students['STUDENTNAME'] == 'Rine Sam']

# s2 = df_student_enrol2['stuCardID']
# s2.isna().sum()
# l2 = list(s2)

# df_student_enrol2[df_student_enrol2['stuCardID'] == 'MH035753']
# df_student_enrol2[df_student_enrol2['Student'] == 'Rine Sam']
# df_student_enrol2[df_student_enrol2['STUDENTNAME'] == 'Rine Sam']

#common_cols = list(set(df_exams_and_students.columns) & set(df_student_enrol2.columns))
#common_cols

#pd.merge(df_exams_and_students, df_student_enrol2, how='right', left_on='STUDENTNAME', right_on='Student')

# Python code t get difference of two lists
# Using set()
# def Diff(li1, li2):
#     return (list(list(set(li1)-set(li2)) + list(set(li2)-set(li1))))
 
# # Driver Code
# li1 = [10, 15, 20, 25, 30, 35, 40]
# li2 = [25, 40, 35]
# print(Diff(l, l2))

In [10]:
# resources http://jonathansoma.com/lede/algorithms-2017/classes/fuzziness-matplotlib/fuzzing-matching-in-pandas-with-fuzzywuzzy/
# Scores: 100 is 100% matching
#print(fuzz.ratio("ghislain hachey", "gislain hachey")) # compares entire string in order
#print(fuzz.partial_ratio("ghislain timbasal", "ghislain hachey")) # compare subsection of the string
#print(fuzz.token_sort_ratio("ghislain hachey", "hachey ghislain")) # ignores work order
#print(fuzz.token_sort_ratio("ghislain hachey", "hachey gislain")) # ignores work order
#print(fuzz.token_set_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")) # ignore duplicate words

# fuzzy on  a dataset
#choices = ['fuzzy fuzzy was a bear', 'is this a test', 'THIS IS A TEST!!']
#process.extract("this is a test", choices, scorer=fuzz.ratio)

# def fuzzy_merge(df1, df2, key1, key2, threshold=90, limit=2):
#     """
#     :param df1: the left table to join
#     :param df2: the right table to join
#     :param key1: key column of the left table
#     :param key2: key column of the right table
#     :param threshold: how close the matches should be to return a match, based on Levenshtein distance
#     :param limit: the amount of matches that will get returned, these are sorted high to low
#     :return: dataframe with boths keys and matches
#     """
#     s = df2[key2].tolist()

#     m = df1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
#     df_1['matches'] = m

#     m2 = df1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
#     df1['matches'] = m2

#     return df1

#df_exams.merge(df_student_enrol, how='left', left_on='STUDENTNAME', right_on='Student', suffixes=('_from_exams', '_from_db'))
#fuzzy_merge(df_exams, df_student_enrol, 'STUDENTNAME', 'Student', 80)

In [11]:
%%time

# Let's get try a bit of fuzzy search see if we can get more students matching
# This fuzzy_pandas package is seamingly more straight forward
import fuzzy_pandas as fpd

exams_cols = list(set(df_exams.columns))
stuen_cols = list(set(df_student_enrol.columns))

df_fuzzy_matches = fpd.fuzzy_merge(
    df_exams, df_student_enrol,
    left_on=['STUDENTNAME'], right_on=['Student'],
    #keep='all',
    method='levenshtein',
    threshold=0.9,
    ignore_case=True,
    ignore_nonalpha=False,
    ignore_nonlatin=False,
    ignore_order_words=False,
    ignore_order_letters=False,
    ignore_titles=False,
    join='left-outer' # { 'inner', 'left-outer', 'right-outer', 'full-outer' }
)

df_fuzzy_matches

#s = df_fuzzy_matches['stuCardID'] == ''
#s.sum()

CPU times: user 42.4 s, sys: 0 ns, total: 42.4 s
Wall time: 42.5 s


,STUDENTID,SPED,ACCOM,STUDENTNAME,SCHOOLID,GENDER,TESTID,TEACHERNAME,ITEM_001,ITEM_002,...,ITEM_030,ITEM_031,ITEM_032,END,stuCardID,Student,stuGender,stuDoB,schNo,stueYear
0,1.0,NO,NO,biten tarkij,NaN,NaN,M03,Imelda Schutz,A,A,...,D,D,D,NaN,MH003302,biten tarkij,M,2010-06-14,MAJ105,2019
1,3.0,NO,NO,keity william,NaN,NaN,M03,Romine Joel,A,D,...,A,B,D,NaN,MH001550,keithy william,F,2009-06-05,UTR101,2019
2,4.0,NO,NO,samson kios,NaN,NaN,M03,Romine Joel,A,C,...,C,C,B,NaN,MH029442,samson kios,M,2009-09-13,UTR101,2019
3,5.0,NO,NO,cuttyno zilla,NaN,NaN,M03,Romine Joel,D,C,...,C,A,B,NaN,MH029441,cuttyno zilla,M,2010-07-05,UTR101,2019
4,6.0,NO,NO,kimilynn kenes,NaN,NaN,M03,Romine Joel,A,C,...,C,A,B,NaN,MH001549,kimilynn kenes,F,2009-11-24,UTR101,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,777.0,NO,NO,riti tobi,NaN,NaN,M03,Akiko Lakjohn,A,C,...,B,C,D,NaN,,,,,,
826,784.0,NO,NO,robert shelenn,NaN,NaN,M03,Takmor Johnson,A,C,...,C,C,D,NaN,,,,,,
827,786.0,NO,NO,spori,NaN,NaN,M03,Takmor Johnson,A,B,...,BLANK,BLANK,BLANK,NaN,,,,,,
828,788.0,NO,NO,jr john,NaN,NaN,M03,Takmor Johnson,C,B,...,B,B,C,NaN,,,,,,


In [12]:
# Repackage the data into the original format for the exams system
df_fuzzy_matches

df_fuzzy_cleaned = df_fuzzy_matches.drop(['SCHOOLID','GENDER','stueYear'] , axis='columns')
df_fuzzy_cleaned.rename(columns={'STUDENTID':'STUDENTID_ORIG','stuCardID':'STUDENTID','stuGender':'GENDER','schNo':'SCHOOLID'}, inplace=True)
df_fuzzy_cleaned = df_fuzzy_cleaned[[
    'STUDENTID_ORIG', 'STUDENTID', 'SPED', 'ACCOM', 'STUDENTNAME', 'Student', 'stuDoB', 'SCHOOLID', 'GENDER', 'TESTID', 'TEACHERNAME', 
    'ITEM_001', 'ITEM_002', 'ITEM_003', 'ITEM_004',
    'ITEM_005', 'ITEM_006', 'ITEM_007', 'ITEM_008', 'ITEM_009', 'ITEM_010',
    'ITEM_011', 'ITEM_012', 'ITEM_013', 'ITEM_014', 'ITEM_015', 'ITEM_016',
    'ITEM_017', 'ITEM_018', 'ITEM_019', 'ITEM_020', 'ITEM_021', 'ITEM_022',
    'ITEM_023', 'ITEM_024', 'ITEM_025', 'ITEM_026', 'ITEM_027', 'ITEM_028',
    'ITEM_029', 'ITEM_030', 'ITEM_031', 'ITEM_032', 'END']]

# minor cleanup of teacher names: at least remove white spaces :)
df_fuzzy_cleaned['TEACHERNAME'] = df_fuzzy_cleaned['TEACHERNAME'].str.strip()
#df_teacher_school[df_teacher_school['TEACHERNAME'] == 'Jiem Lakmej']

# Re camel case student names
df_fuzzy_cleaned['STUDENTNAME']= df_fuzzy_cleaned['STUDENTNAME'].str.title()
df_fuzzy_cleaned['Student']= df_fuzzy_cleaned['Student'].str.title()

df_fuzzy_cleaned.sort_values(by=['STUDENTID_ORIG'])
#df_fuzzy_cleaned.columns
#len(df_fuzzy_cleaned['STUDENTID_ORIG'].unique())

,STUDENTID_ORIG,STUDENTID,SPED,ACCOM,STUDENTNAME,Student,stuDoB,SCHOOLID,GENDER,TESTID,...,ITEM_024,ITEM_025,ITEM_026,ITEM_027,ITEM_028,ITEM_029,ITEM_030,ITEM_031,ITEM_032,END
0,1.0,MH003302,NO,NO,Biten Tarkij,Biten Tarkij,2010-06-14,MAJ105,M,M03,...,C,D,D,D,D,D,D,D,D,NaN
599,2.0,,NO,NO,Ainet Milne,,,,,M03,...,A,D,A,C,B,A,C,B,D,NaN
1,3.0,MH001550,NO,NO,Keity William,Keithy William,2009-06-05,UTR101,F,M03,...,D,D,A,C,A,D,A,B,D,NaN
2,4.0,MH029442,NO,NO,Samson Kios,Samson Kios,2009-09-13,UTR101,M,M03,...,B,D,A,C,A,A,C,C,B,NaN
3,5.0,MH029441,NO,NO,Cuttyno Zilla,Cuttyno Zilla,2010-07-05,UTR101,M,M03,...,B,D,A,B,A,A,C,A,B,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,790.0,MH010443,NO,NO,Bernice Subille,Bernice Subille,2010-05-14,RON101,F,M03,...,A,D,B,C,A,A,C,D,A,NaN
829,791.0,,NO,NO,Deied Dribo,,,,,M03,...,D,D,A,B,C,D,C,B,A,NaN
596,792.0,MH010787,NO,NO,Ailynn Robe,Ailynn Robe,2010-05-03,KWA102,F,M03,...,A,B,A,B,A,B,MULT,MULT,MULT,NaN
597,793.0,MH010784,NO,NO,Jolynn Sailass,Joylynn Sailass,2009-11-02,KWA102,F,M03,...,A,B,A,B,A,B,A,B,A,NaN


In [14]:
# Now try some educated 'guesses'. Set the school based on known variable:
# the teachers' most commonly supervised school
df_teacher_school = df_fuzzy_cleaned[['SCHOOLID','TEACHERNAME']].copy()

# All teachers with known schools based on fuzzy search on students in EMIS data
df_teacher_with_school = df_teacher_school[df_teacher_school['SCHOOLID'] != ''].copy()

# All students remaining with no schools (or gender) known from fuzzy searching EMIS data
df_teacher_no_school = df_teacher_school[df_teacher_school['SCHOOLID'] == ''].copy()

#df_teacher_with_school#.groupby(by='TEACHERNAME')
df_teacher_with_school_occurance = df_teacher_with_school.value_counts(sort=True, ascending=False)
df_teacher_with_school_occurance = df_teacher_with_school_occurance.reset_index().groupby('TEACHERNAME').first()
del df_teacher_with_school_occurance[0]
teacher_school = df_teacher_with_school_occurance.to_dict()['SCHOOLID']
teacher_school

{'?': 'MAJ125',
 'AJ': 'KWA108',
 'Ailiasi Koroi': 'MAJ116',
 'Akiko Lakjohn': 'WTH103',
 'Alee Langbon': 'KIL101',
 'Amelia Arno': 'MIL105',
 'Arimon': 'JAL105',
 'Arthur Johnny': 'KWA111',
 'Betty Ajen': 'MAJ109',
 'Blenda Lontak': 'ALU101',
 'Brenda Samuel': 'NAM101',
 'Carthy Hosia': 'EBO103',
 'Clanny Carlon': 'KWA108',
 'Clena Bujen': 'KWA102',
 'Connie': 'ARN101',
 'Danette Wayne': 'MAJ103',
 'Dolores Jabnil': 'MAJ109',
 'Elisa Samson': 'AIL105',
 'Elsai Jokray': 'MAJ110',
 'Emi Term': 'MAJ112',
 'Grace Stephen': 'KWA115',
 'Hellen Azaredo': 'MAJ107',
 'Hidle Kilma': 'NAU103',
 'Imelda Schutz': 'MAJ105',
 'Isaac Lojan': 'ARN109',
 'Jemina Jai': 'AIL102',
 'Jiem Lakmej': 'MAJ127',
 'Joe Lomae': 'JAL109',
 'Jordan Riklon': 'MAJ126',
 'Julian Lelwoj': 'MAL105',
 'Julie Bernaldo': 'KWA108',
 'Junior Aine': 'ARN104',
 'Kannea L': 'MAJ126',
 'Katol Levai': 'MAJ101',
 'Komja B': 'AIL107',
 'Latter Ben': 'MAJ105',
 'Laura Fredrik': 'LIK103',
 'Leivie Martinez': 'KWA108',
 'Lessie Abner'

In [15]:
# Start by assigning a best guess school to all remaining records without one
# However, only do this on the subset of rows with no acquired 
# information from the EMIS DB
df_fuzzy_cleaned[['SCHOOLID','TEACHERNAME']]
df_fuzzy_cleaned.loc[df_fuzzy_cleaned['SCHOOLID'] == '', ['SCHOOLID']] = df_fuzzy_cleaned['TEACHERNAME'].map(teacher_school)
df_fuzzy_cleaned

,STUDENTID_ORIG,STUDENTID,SPED,ACCOM,STUDENTNAME,Student,stuDoB,SCHOOLID,GENDER,TESTID,...,ITEM_024,ITEM_025,ITEM_026,ITEM_027,ITEM_028,ITEM_029,ITEM_030,ITEM_031,ITEM_032,END
0,1.0,MH003302,NO,NO,Biten Tarkij,Biten Tarkij,2010-06-14,MAJ105,M,M03,...,C,D,D,D,D,D,D,D,D,NaN
1,3.0,MH001550,NO,NO,Keity William,Keithy William,2009-06-05,UTR101,F,M03,...,D,D,A,C,A,D,A,B,D,NaN
2,4.0,MH029442,NO,NO,Samson Kios,Samson Kios,2009-09-13,UTR101,M,M03,...,B,D,A,C,A,A,C,C,B,NaN
3,5.0,MH029441,NO,NO,Cuttyno Zilla,Cuttyno Zilla,2010-07-05,UTR101,M,M03,...,B,D,A,B,A,A,C,A,B,NaN
4,6.0,MH001549,NO,NO,Kimilynn Kenes,Kimilynn Kenes,2009-11-24,UTR101,F,M03,...,B,D,A,B,A,A,C,A,B,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,777.0,,NO,NO,Riti Tobi,,,WTH103,,M03,...,C,C,B,D,A,A,B,C,D,NaN
826,784.0,,NO,NO,Robert Shelenn,,,RON101,,M03,...,A,C,C,B,C,C,C,C,D,NaN
827,786.0,,NO,NO,Spori,,,RON101,,M03,...,BLANK,D,C,B,D,BLANK,BLANK,BLANK,BLANK,NaN
828,788.0,,NO,NO,Jr John,,,RON101,,M03,...,B,A,B,C,B,B,B,B,C,NaN


In [16]:
# Write processed data back into excel for final scrutiny
f_cleaned = os.path.join(cwd, 'data/RMI/poon-cleanup-request/M03 2019-cleaned.xlsx')
df_fuzzy_cleaned.to_excel(f_cleaned, index=False, sheet_name='M03 2019', engine='openpyxl')